In [1]:
import os
%pwd

'c:\\Users\\nikil\\Documents\\Project\\DL Projects\\DL_one\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\nikil\\Documents\\Project\\DL Projects\\DL_one'

Entitity : Return type of any function 
In this case the data ingestion function

In [5]:


from dataclasses import dataclass
from pathlib import Path

#returns from congif.yaml
#dataclass makes it a variable that can be accessed from other files as well
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
#update config in src config manager



class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    #defined in the previous code and will return only the below return
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config



In [8]:
#update data ingestion component
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 27)

In [12]:
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [13]:
#update the pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-07 21:39:34,615: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-07 21:39:34,646: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-07 21:39:34,653: INFO: common: created directory at: artifacts]
[2023-12-07 21:39:34,658: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-07 21:39:34,663: INFO: 773878808: Downloading data from https://drive.google.com/file/d/1Cvoh9q62G3yixfxbed3PmDCylj03cD-2/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1Cvoh9q62G3yixfxbed3PmDCylj03cD-2
From (redirected): https://drive.google.com/uc?/export=download&id=1Cvoh9q62G3yixfxbed3PmDCylj03cD-2&confirm=t&uuid=7f5f92d9-fa3c-448b-9203-63e646470a6c
To: c:\Users\nikil\Documents\Project\DL Projects\DL_one\artifacts\data_ingestion\data.zip
100%|██████████| 472M/472M [05:48<00:00, 1.35MB/s] 

[2023-12-07 21:45:26,606: INFO: 773878808: Downloaded data from https://drive.google.com/file/d/1Cvoh9q62G3yixfxbed3PmDCylj03cD-2/view?usp=sharing into file artifacts/data_ingestion/data.zip]


: 